In [67]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.metrics import mean_squared_error
import torch.optim.lr_scheduler as lr_scheduler
from sympy.core.random import shuffle
from torch.utils.data import DataLoader, TensorDataset
import math
import src.generate_encodings as ge
import random
import torch.optim as optim
import time
import datetime
from tqdm import tqdm

In [68]:
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

True


'NVIDIA GeForce RTX 4060 Ti'

In [69]:
data_set = "../Data/Protein_Gym_Datasets/HIS7_YEAST_Pokusaeva_2019.csv"

to_encode = []
with open(data_set, "r") as data_file:
    lines = data_file.readlines()[1:]
for line in lines:
    to_encode.append(line.split(",")[1])

In [70]:
data_pairs = []
embeddings = ge.load_encodings("../Data/Embeddings/HIS7_YEAST_Pokusaeva_2019/esm1b")

print("embeddings loaded")
for i, line in enumerate(lines):
#     embedding = ge.generate_sequence_encodings("georgiev", [line.split(",")[1]])[0]
    score = line.split(",")[2]
    embedding = embeddings[i]
    data_pairs.append((embedding, float(score)))
# del embeddings
print(data_pairs[0])
print(data_pairs[1])

IsADirectoryError: [Errno 21] Is a directory: '../Data/Embeddings/HIS7_YEAST_Pokusaeva_2019/esm1b/batch_001'

In [64]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [65]:
random.shuffle(data_pairs)

split_ratio = (0.75, 0.15, 0.10)
split_train = int(len(data_pairs) * split_ratio[0])
split_valid = int(len(data_pairs) * (split_ratio[1] + split_ratio[0]))

x_train = torch.tensor([xy[0].cpu().reshape(-1) for xy in data_pairs[0:split_train]], dtype=torch.float32)
x_valid = torch.tensor([xy[0].cpu().reshape(-1) for xy in data_pairs[split_train:split_valid]],
                       dtype=torch.float32)
x_test = torch.tensor([xy[0].cpu().reshape(-1) for xy in data_pairs[split_valid:]], dtype=torch.float32)

y_train = torch.tensor([[float(xy[1])] for xy in data_pairs[0:split_train]], dtype=torch.float32)
y_valid = torch.tensor([[float(xy[1])] for xy in data_pairs[split_train: split_valid]], dtype=torch.float32)
y_test = torch.tensor([[float(xy[1])] for xy in data_pairs[split_valid:]], dtype=torch.float32)

# x_train = torch.tensor(np.array([xy[0].cpu().reshape(-1) for xy in data_pairs[0:split_train]]), dtype=torch.float32)
# x_valid = torch.tensor(np.array([xy[0].cpu().reshape(-1) for xy in data_pairs[split_train:split_valid]]),
#                        dtype=torch.float32)
# x_test = torch.tensor(np.array([xy[0].cpu().reshape(-1) for xy in data_pairs[split_valid:]]), dtype=torch.float32)
#
# y_train = torch.tensor(np.array([[float(xy[1])] for xy in data_pairs[0:split_train]]), dtype=torch.float32)
# y_valid = torch.tensor(np.array([[float(xy[1])] for xy in data_pairs[split_train: split_valid]]), dtype=torch.float32)
# y_test = torch.tensor(np.array([[float(xy[1])] for xy in data_pairs[split_valid:]]), dtype=torch.float32)

train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=1000, shuffle=False)

test_data = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

valid_data = TensorDataset(x_valid, y_valid)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

input_len = len(x_train[0])

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [58]:
class FFNet(nn.Module):
    def __init__(self):
        super(FFNet, self).__init__()

        self.fc1 = nn.Linear(input_len, input_len - 1)
        self.fc2 = nn.Linear(input_len - 1, input_len)
        self.fc3 = nn.Linear(input_len, input_len - 1)
        self.fc4 = nn.Linear(input_len - 1, input_len)
        self.fc5 = nn.Linear(input_len, input_len - 1)
        self.fc6 = nn.Linear(input_len - 1, input_len)
        self.fc7 = nn.Linear(input_len, input_len - 1)
        self.fc8 = nn.Linear(input_len - 1, input_len)
        self.fc9 = nn.Linear(input_len, input_len - 1)
        self.fc10 = nn.Linear(input_len - 1, input_len)
        self.fc11 = nn.Linear(input_len, input_len - 1)
        self.output_layer = nn.Linear(input_len - 1, 1)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))
        x = torch.relu(self.fc8(x))
        x = torch.relu(self.fc9(x))
        x = torch.relu(self.fc10(x))
        # x = torch.relu(self.fc11(x))


        x = self.output_layer(x)
        return x




In [59]:
gt_model = FFNet()
criterion = nn.MSELoss()

learning_rate = 0.001
momentum = 0.9
optim = "SGD"
if optim == "SGD":
    optimizer = torch.optim.SGD(gt_model.parameters(), lr=learning_rate, momentum=momentum)
else:
    optimizer = torch.optim.Adam(gt_model.parameters(), lr=learning_rate / 100)
# scheduler = lr_scheduler.LRScheduler(optimizer)

In [60]:
"""Track Training Metrics"""
time_stamp = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M")
outfile = f"../Models/Logs/{time_stamp}.txt"
with open(outfile, "w") as f:
    f.write(f"Optimizer: {optim}, learning_rate: {learning_rate}, momentum: {momentum} \n")
    f.write(f"Model: \n {gt_model} \n")
    f.write("\n")

In [61]:
n_epochs = 500
diplay_every_n_epochs = 1

for epoch in range(n_epochs):
    start_time = time.time()
    gt_model.train()
    running_loss = 0.0
    tr_y_pred = []
    tr_y_true = []

    best_val = 1
    best_model = None

    for tr_inputs, tr_labels in train_loader:
        optimizer.zero_grad()
        tr_outputs = gt_model.forward(tr_inputs)
        loss = criterion(tr_outputs, tr_labels)
        running_loss += loss.item()
        tr_y_pred.append(tr_outputs)
        tr_y_true.append(tr_labels)
        loss.backward()
        optimizer.step()

    # scheduler.step()

    gt_model.eval()
    with torch.no_grad():
        te_loss = 0.0
        running_va_loss = 0.0

        va_y_pred = []
        va_y_true = []

        for va_input, va_label in valid_loader:
            va_outputs = gt_model.forward(va_input)
            va_loss = criterion(va_outputs, va_label)
            running_va_loss += va_loss.item()
            va_y_pred.append(va_outputs)
            va_y_true.append(va_label)

        tr_y_true = torch.cat(tr_y_true).detach().numpy()
        tr_y_pred = torch.cat(tr_y_pred).detach().numpy()
        tr_RMSE = math.sqrt(mean_squared_error(tr_y_true, tr_y_pred))

        va_y_true = torch.cat(va_y_true).detach().numpy()
        va_y_pred = torch.cat(va_y_pred).detach().numpy()
        va_RMSE = math.sqrt(mean_squared_error(va_y_true, va_y_pred))

    with open(outfile, "a") as f:
        f.write(
            f'[Epoch {epoch + 1}/{n_epochs}, t: {round(time.time() - start_time, 2)}s]:Train Loss: {running_loss / len(train_loader):.4f}, '
            f'Train RMSE: {tr_RMSE:.4f}, Valid Loss: {running_va_loss / len(valid_loader):.4f}, Valid RMSE: {va_RMSE:.4f} \n')

    if (epoch + 1) % diplay_every_n_epochs == 0:
        print(
            f'[Epoch {epoch + 1}/{n_epochs}, t: {round(time.time() - start_time, 2)}s]:Train Loss: {running_loss / len(train_loader):.4f}, '
            f'Train RMSE: {tr_RMSE:.4f}, Valid Loss: {running_va_loss / len(valid_loader):.4f}, Valid RMSE: {va_RMSE:.4f} ')

[Epoch 1/500, t: 49.37s]:Train Loss: 0.2254, Train RMSE: 0.4749, Valid Loss: 0.2021, Valid RMSE: 0.4495 
[Epoch 2/500, t: 50.45s]:Train Loss: 0.2015, Train RMSE: 0.4489, Valid Loss: 0.2021, Valid RMSE: 0.4495 
[Epoch 3/500, t: 50.45s]:Train Loss: 0.2015, Train RMSE: 0.4489, Valid Loss: 0.2021, Valid RMSE: 0.4495 


KeyboardInterrupt: 